<a href="https://colab.research.google.com/github/DiegoJimenezTamame/GitHub_tutor/blob/main/train_musicGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# os.environ['LC_ALL'] = 'C.UTF-8'
# os.environ['LANG'] = 'C.UTF-8'

In [2]:
!pip install pydub

In [3]:
!python3 -m pip install -U git+https://github.com/facebookresearch/audiocraft#egg=audiocraft


  Cloning https://github.com/facebookresearch/audiocraft to /tmp/pip-install-5nn0hash/audiocraft_55841ab89de44e0b940e8b2b4610f09d
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/audiocraft /tmp/pip-install-5nn0hash/audiocraft_55841ab89de44e0b940e8b2b4610f09d
  Resolved https://github.com/facebookresearch/audiocraft to commit e5fcc458a4dc1c6f7248cbceac9cfe471f2c92b8
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 71.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 55.8 MB/s eta 0:00:00
  Preparing m

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
import random
from pydub import AudioSegment

import soundfile as sf
import numpy as np

In [6]:
# Folder names
input_folder_name = "raw"
output_folder_name = "output"
musicgen_trainer_dir = "musicgen_trainer_dir"

if not os.path.exists(input_folder_name):
    os.makedirs(input_folder_name)
    print(f"'{input_folder_name}' folder created.")

if not os.path.exists(output_folder_name):
    os.makedirs(output_folder_name)
    print(f"'{output_folder_name}' folder created.")

if not os.path.exists(musicgen_trainer_dir):
    os.makedirs(musicgen_trainer_dir)
    print(f"'{musicgen_trainer_dir}' folder created.")

'raw' folder created.
'output' folder created.
'musicgen_trainer_dir' folder created.


In [11]:
def delete_files_in_directory(directory_path):
    # Iterate through the files in the directory
    for file_name in os.listdir(directory_path):
        # Construct the full file path
        file_path = os.path.join(directory_path, file_name)

        # Check if it is a file (not a directory)
        if os.path.isfile(file_path):
            # Delete the file
            os.remove(file_path)
            print(f"Deleted: {file_path}")

# Specify the output directory where you want to delete files
output_directory = 'output'

# Delete all files in the output directory
delete_files_in_directory(output_directory)

In [ ]:
import shutil

# Specify the directory you want to delete
directory_path = "/content/musicgen_trainer_dir"

# Delete the directory and all its contents
shutil.rmtree(directory_path)

In [12]:

def process_audio(file_path, output_dir, segment_length=30):
    # Load audio file
    audio = AudioSegment.from_file(file_path)

    # Get file name without extension for caption
    file_name = os.path.splitext(os.path.basename(file_path))[0]

    # Convert segment length to milliseconds
    segment_length_ms = segment_length * 1000

    # Set the sample rate to 32000 Hz
    audio = audio.set_frame_rate(32000)

    # Calculate the number of segments
    num_segments = (len(audio) + segment_length_ms - 1) // segment_length_ms

    # Loop through segments
    for i in range(num_segments):
        # Get start time for the segment
        start_time = i * segment_length_ms

        # If this is the last segment, adjust start_time
        if i == num_segments - 1:
            start_time = len(audio) - segment_length_ms

        # Get end time for the segment
        end_time = start_time + segment_length_ms

        # Extract the segment
        segment = audio[start_time:end_time]

        # Save the segment
        segment.export(os.path.join(output_dir, f'segment_{i:03d}.wav'), format='wav')

        # Save the caption
        with open(os.path.join(output_dir, f'segment_{i:03d}.txt'), 'w') as f:
            f.write(file_name)

# Directory setup
output_directory = 'output'
samples_directory = 'raw'

# Check if the output directory exists, if not, create it
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Iterate through the files in the "samples" directory
for file_name in os.listdir(samples_directory):
    if file_name.endswith('.wav') or file_name.endswith('.mp3'):
        file_path = os.path.join(samples_directory, file_name)
        process_audio(file_path, output_directory, segment_length=30)


In [13]:

import librosa

# Directory where the WAV files are saved
output_directory = 'output'

# Iterate through the files in the directory
for file_name in os.listdir(output_directory):
    # Check if the file is a WAV file
    if file_name.endswith('.wav'):
        # Load the audio file
        file_path = os.path.join(output_directory, file_name)
        audio, sample_rate = librosa.load(file_path, sr=None)

        # Check the shape of the audio
        if audio.shape[0] == 32000 * 30:
            print(f"{file_name} has the correct shape: {audio.shape[0]}")
        else:
            print(f"{file_name} does not have the correct shape. Actual shape: {audio.shape[0]}")


segment_001.wav has the correct shape: 960000
segment_000.wav has the correct shape: 960000


#train

In [14]:
!cd "/content/musicgen_trainer_dir" && git clone https://github.com/chavinlo/musicgen_trainer.git

Cloning into 'musicgen_trainer'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 166 (delta 3), reused 2 (delta 2), pack-reused 162 (from 1)
Receiving objects: 100% (166/166), 10.53 MiB | 15.42 MiB/s, done.
Resolving deltas: 100% (97/97), done.


In [15]:
!python /content/musicgen_trainer_dir/musicgen_trainer/run.py --dataset_path /content/output

2025-01-21 21:21:17.755629: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-21 21:21:18.037641: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-21 21:21:18.115417: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-21 21:21:18.558049: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-21 21:21:20.805723: W tensorflow/comp

In [16]:
from audiocraft.models import musicgen
from audiocraft.utils.notebook import display_audio
import torch

In [23]:
model = musicgen.MusicGen.get_pretrained('small', device='cuda')

model.set_generation_params(duration=20)

model.lm.load_state_dict(torch.load('models/lm_final.pt'))

/usr/local/lib/python3.10/dist-packages/audiocraft/models/musicgen.py:83: UserWarning: MusicGen pretrained model relying on deprecated checkpoint mapping. Please use full pre-trained id instead: facebook/musicgen-small
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


<All keys matched successfully>

In [ ]:
res = model.generate([
    'experimental drum solo',
    'unpitched percussion solo, dark, slow, textured',
    'fast breakdown, metal, grindcore',
    'improvisatory, fast, no pitch, drumrolls'
],
    progress=True)
display_audio(res, 32000)